In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [15]:
titles = []
years = []
time = []
imdb_ratings = []
metascore = []
votes = []
us_gross = []

In [16]:
base_url = "https://www.imdb.com"
second_url = "/search/title/?groups=top_1000"
headers = {"Accept-Language" : "en-US, en;q=0.5"}

In [17]:
while True:
    url = base_url + second_url
    results = requests.get(url, headers=headers)
    soup = BeautifulSoup(results.text, "html.parser")

    movie_div = soup.find_all('div', class_='lister-item mode-advanced')

    for container in movie_div:
        #name
        name = container.h3.a.text
        titles.append(name)
        
        #year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        #runtime
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        #IMDB rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        #metascore
        meta = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascore.append(meta)
        
        #votes and grosses
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv)>1 else '-'
        us_gross.append(grosses)
        
    try:
        second_url = soup.find('a', class_='lister-page-next').get('href')
    except:
        print('No More url')
        break

No More url


In [18]:
movies = pd.DataFrame({
'movie':titles,
'year':years,
'timeMin':time,
'imdb':imdb_ratings,
'metascore':metascore,
'votes':votes,
'us_grossMillions':us_gross,
})

In [22]:
movies.dtypes

movie                object
year                 object
timeMin              object
imdb                float64
metascore            object
votes                object
us_grossMillions     object
dtype: object

In [42]:
# movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
# movies['timeMin'] = movies['timeMin'].str.extract('(\d+)').astype(int)
# movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')
# movies['votes'] = movies['votes'].str.replace(',','').astype(int)
# movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
# movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')
movies.dtypes

movie                object
year                  int32
timeMin               int32
imdb                float64
metascore           float64
votes                 int32
us_grossMillions    float64
dtype: object